План проекта:
Изучить файл с данными.
Разбить данные на три выборки: обучающая, валидационная и тестовая.
Исследовать три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
Найти оптимальные параметры для каждой модели и выбрать одну из них для обучения модели.
Оценить точность обученной модели.
Оценить адекватность модели.

### Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

### Открываем файл с данными

In [2]:
df = pd.read_csv('users_behavior.csv')

### Изучим данные

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Файл данных содержит 3214 записей, что не так много. Модель может быть несколько переобученной.\
Целевым признаком у нас будет "is_ultra". \
Целевой признак категориальный, поэтому решаем задачу классификации. \
Категорий всего две — поэтому речь идёт о бинарной (двоичной) классификации.

## Разбейте данные на выборки

### Делим выборку на данные на обучающую, валидационную и тестовую выборки в соотношении 60\20\20

In [5]:
df_train, aux_df_valid = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])
df_valid, df_test = train_test_split(aux_df_valid, test_size=0.5, random_state=12345, stratify=aux_df_valid['is_ultra'])

In [6]:
print('Размер обучающей выборки', df_train.shape[0])
print('Размер валидационной выборки', df_valid.shape[0])
print('Размер тестовой выборки', df_test.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


### Создаем отдельные переменные для признаков

In [7]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Выводы по второму этапу. \
Разделили данные на обучающие, валидационные и тестовые в пропорции 60/20/20/. \
Обучение моделей проведем на features_train, проверим работу на features_valid. \
Лучшую модель проверим на features_test.

## Исследуйте модели

### Модель - решающее дерево

Основной гиперпараметр решающего дерева - глубина. \
Перебираем в алгоритме обучения разные варианты глубины дерева.

In [8]:
best_model_tree = None
best_result_tree = 0
for depth in range(1, 25):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_tree.fit(features_train, target_train) 
    predictions_tree = model_tree.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_tree) 
    print(depth, result)
    if result > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result
print("Модель:", best_model_tree, "Accuracy лучшей модели решающего дерева:", best_result_tree)

1 0.7402799377916018
2 0.7729393468118196
3 0.7776049766718507
4 0.7542768273716952
5 0.7853810264385692
6 0.7744945567651633
7 0.7869362363919129
8 0.80248833592535
9 0.7822706065318819
10 0.7729393468118196
11 0.7620528771384136
12 0.7527216174183515
13 0.744945567651633
14 0.7558320373250389
15 0.7480559875583204
16 0.7387247278382582
17 0.7153965785381027
18 0.7076205287713841
19 0.7045101088646968
20 0.7060653188180405
21 0.7013996889580093
22 0.7013996889580093
23 0.7013996889580093
24 0.7013996889580093
Модель: DecisionTreeClassifier(max_depth=8, random_state=12345) Accuracy лучшей модели решающего дерева: 0.80248833592535


Лучший показатель при глубине дерева 3.
Попробуем изменить гиперпараметр criterion с gini на entropy и splitter с best на random

In [9]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=3, criterion="entropy") 
model_tree.fit(features_train, target_train) 
predictions = model_tree.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
print("Глубина дерева", 3, "Accuracy", result)

Глубина дерева 3 Accuracy 0.7776049766718507


In [10]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3, splitter="random") 
model.fit(features_train, target_train) 
predictions = model.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
print("Глубина дерева", 3, "Accuracy", result)

Глубина дерева 3 Accuracy 0.7262830482115086


Гиперпараметры по умолчанию показывают лучший результат

### Модель - случайный лес

Основной гиперпараметр случайного леса - количество деревьев.
Перебираем в алгоритме обучения разные варианты. 

In [11]:
est_e = 0
best_model_forest = None
best_result_forest = 0
for est in range(1, 50):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model_forest.fit(features_train, target_train) 
    result = model_forest.score(features_valid, target_valid) 
    print(est, result)
    if result > best_result_forest:
        est_e = est
        best_model_forest = model_forest
        best_result_forest = result
print("Количество решающих деревьев:", est_e, "Accuracy лучшей модели случайного леса:", best_result_forest) 

1 0.6998444790046656
2 0.7433903576982893
3 0.7418351477449455
4 0.7558320373250389
5 0.7527216174183515
6 0.7729393468118196
7 0.7698289269051322
8 0.776049766718507
9 0.7698289269051322
10 0.7869362363919129
11 0.7807153965785381
12 0.7853810264385692
13 0.7884914463452566
14 0.7869362363919129
15 0.7884914463452566
16 0.7869362363919129
17 0.7916018662519441
18 0.7916018662519441
19 0.7931570762052877
20 0.7916018662519441
21 0.7993779160186625
22 0.7993779160186625
23 0.7947122861586314
24 0.7978227060653188
25 0.7947122861586314
26 0.7931570762052877
27 0.7947122861586314
28 0.7993779160186625
29 0.7916018662519441
30 0.7931570762052877
31 0.7931570762052877
32 0.7931570762052877
33 0.7884914463452566
34 0.7931570762052877
35 0.7931570762052877
36 0.7900466562986003
37 0.7916018662519441
38 0.7884914463452566
39 0.7931570762052877
40 0.7931570762052877
41 0.8009331259720062
42 0.8009331259720062
43 0.7993779160186625
44 0.7962674961119751
45 0.8009331259720062
46 0.80248833592535


### Модель - логистическая регрессия

In [12]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regression.fit(features_train, target_train) 
model_regression.predict(features_valid)
result_regression = model.score(features_valid, target_valid) 
print("Accuracy модели логистической регрессии:", result)

Accuracy модели логистической регрессии: 0.7993779160186625


По результатам проверки лучшим алгоритмом признаем алгоритм классификации — случайный лес (англ. random forest) с количеством решающих деревьев 22.

## Проверьте модель на тестовой выборке

In [13]:
#predictions_tree_test = model_tree.predict(features_test) 
#result = accuracy_score(target_test, predictions_tree_test)
#print("Accuracy модели решающего дерева на тестовой выборке:", result)

In [14]:
best_model_forest = RandomForestClassifier(random_state=12345, n_estimators=22) 
best_model_forest.fit(features_train, target_train) 
predictions_forest_test = best_model_forest.predict(features_test)
result = best_model_forest.score(features_test, target_test) 
print("Accuracy модели случайного леса на тестовой выборке:", result)

Accuracy модели случайного леса на тестовой выборке: 0.80248833592535


In [15]:
#predictions_regression_test = model_regression.predict(features_test) 
#result = accuracy_score(target_test, predictions_regression_test)
#print("Accuracy модели логистической регрессии на тестовой выборке:", result)

Тестовая выборка показала правильность выбора модели классификации.

## (бонус) Проверьте модели на адекватность

При создании модели машинного обучения для какого-либо проекта разумно сначала создать baseline модель. Она представляет собой простейшую (dummy) модель, которая всегда предсказывает наиболее часто встречающийся класс. Так вы получите контрольные данные для сравнительной оценки построенной вами более сложной модели. К тому же вы сможете быть уверены в качестве ее работы, например, в том, что она выдает не просто набор случайно подобранных данных.
В библиотеке scikit-learn есть DummyClassifier() для задач классификации.\
Параметром strategy установим:\
“uniform” - этот генерирует прогнозы с учетом распределения классов обучающего набора.\
«most_frequent»: всегда предсказывает наиболее часто встречающуюся метку в обучающем наборе.\
«stratified»: генерирует прогнозы с учетом распределения классов обучающего набора.

In [16]:
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(features_train, target_train)
predictions_adequacy_test = dummy_clf.predict(features_test)
result = accuracy_score(target_test, predictions_adequacy_test)
print("Accuracy адекватности:", result)

Accuracy адекватности: 0.47900466562986005


In [17]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
predictions_adequacy_test = dummy_clf.predict(features_test)
result = accuracy_score(target_test, predictions_adequacy_test)
print("Accuracy адекватности:", result)

Accuracy адекватности: 0.6936236391912908


In [18]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(features_train, target_train)
predictions_adequacy_test = dummy_clf.predict(features_test)
result = accuracy_score(target_test, predictions_adequacy_test)
print("Accuracy адекватности:", result)

Accuracy адекватности: 0.5769828926905132


Полученные случайные прогнозы сильно хуже наших проверенных моделей.

## Общий вывод

В ходе выполнения проекта:

Изучен файл с данными.\
Разбиты данные на три выборки: обучающая, валидационная и тестовая.\
Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.\
Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.\
Оценена точность обученной модели.\
Оценена адекватность модели.\
Для обучения модели была использована выборка в 1928 значений. Этого может быть недостаточно для повышения ключевых параметров модели классификатора.\
Проведена проверка адекватности моделей.